In [ ]:
import numpy as np
from scipy import stats

In [ ]:
## This eval script calculates CIs for all results we run:

def calculate_confidence_interval(
    data:np.array,
    percentage:bool=True,
    round_to:int=3,
    method='t',
    confidence_level=0.95):
    """
    Calculate the mean, standard error, and confidence interval for a mean (Could be extended to any point estimate).

    This function can compute the confidence interval using different statistical methods: t-score, z-score, or bootstrap.
    The results can be expressed as either proportions or percentages, and the output can be rounded to a specified number
    of decimal places.

    Args:
        is_correct (array-like): An array of binary values (0 or 1) indicating correctness.
        percentage (bool, optional): If True, results are returned as percentages (0-100). Defaults to True.
        round_to (int, optional): Number of decimal places to round the results to. Defaults to 3.
        method (str, optional): Statistical method for confidence interval calculation ('t', 'z', or 'bootstrap'). Defaults to 't'.
        confidence_level (float, optional): Confidence level for the interval, expressed as a decimal (e.g., 0.95 for 95% confidence). Defaults to 0.95.

    Returns:
        tuple: A tuple containing the mean, standard error, lower bound of the confidence interval,
               and upper bound of the confidence interval. The format depends on the `percentage` argument.
    """
    # Calculate mean and standard error
    mean      = data.mean()
    std_error = data.std() / np.sqrt(len(data))

    # Confidence interval calculation based on method
    if method == 't':
        # T-distribution
        degrees_freedom = len(data) - 1
        confidence_interval = stats.t.interval(confidence_level, degrees_freedom, mean, std_error)

    elif method == 'z':
        # Z-distribution
        z_value = stats.norm.ppf(1 - (1 - confidence_level) / 2)  # Two-tailed z-value for the confidence level
        confidence_interval = (mean - z_value * std_error, mean + z_value * std_error)

    elif method == 'bootstrap':
        # Bootstrap confidence interval
        n_iterations = 1000
        bootstrapped_means = []
        for _ in range(n_iterations):
            sample = stats.resample(data, replace=True)
            bootstrapped_means.append(sample.mean())
        lower = np.percentile(bootstrapped_means, (1 - confidence_level) / 2 * 100)
        upper = np.percentile(bootstrapped_means, (1 + confidence_level) / 2 * 100)
        confidence_interval = (lower, upper)

    else:
        raise ValueError("Method must be 't', 'z', or 'bootstrap'.")

    # Convert to percentage if required
    if percentage:
        mean *= 100
        confidence_interval = [ci * 100 for ci in confidence_interval]

    # Round results if specified
    if round_to is not None:
        mean = round(mean, round_to)
        confidence_interval = [round(ci, round_to) for ci in confidence_interval]

    out = {"mean":mean,"ci":confidence_interval}
    return out


def format_output(question_type:str,
                  metrics:dict[str,float]) -> None:
    """
    Print formatted output for the given question type and its associated metrics.

    Parameters
    ----------
    question_type : str
        The type of question being processed, which will be capitalized and displayed.
    metrics : dict[str, float]
        A dictionary containing metric information. It must include:
        - 'mean': The mean accuracy to display.
        - 'ci': A list or tuple containing two values representing the confidence interval.

    Returns
    -------
    None
        This function does not return any value; it simply prints the formatted output.
    """
    print(f"{question_type:<36}  Acc: {metrics['mean']:>6.3f} ({metrics['ci'][0]:>4.3f},{metrics['ci'][1]:>4.3f})")
